# **Stock Project**

# **Imports**

In [1]:
import requests
import json
from bs4 import BeautifulSoup

# **Obtain Price Function**

In [2]:
# Import Data
url = 'https://finance.yahoo.com/quote/ES%3DF?p'

In [3]:
r = requests.get(url)

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

In [10]:
# Get Price
price = soup.find('fin-streamer', {'class':'Fw(b) Fz(36px) Mb(-4px) D(ib)'}).text

In [11]:
print(price)

4,131.00


In [13]:
# Get Price 
Pchange = soup.find('fin-streamer', {'class':'Fw(500) Pstart(8px) Fz(24px)'}).text

In [14]:
print(price, Pchange)

4,131.00 +2.00
